In [17]:
import pyomo.environ as pyo
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from dash import Dash, dcc, html
import webbrowser
from threading import Timer
import dash_bootstrap_components as dbc
import plotly.io as pio

# Read price data from Excel
price_data = pd.read_csv('price_data.csv')
price = price_data['Price'].values  # Assuming the second column is named 'Price'


In [4]:
# Constants
hours = len(price)  # Should be 8760 for a full year
max_charge_rate = 1  # kW
max_discharge_rate = 1  # kW
battery_capacity = 4  # kWh
initial_soc = 2  # kWh
final_soc = 2  # kWh
charge_efficiency = 0.9
discharge_efficiency = 0.9

# Create the model
model = pyo.ConcreteModel()

# Time index
model.T = pyo.RangeSet(0, hours - 1)

# Variables
model.charge = pyo.Var(model.T, within=pyo.NonNegativeReals, bounds=(0, max_charge_rate))
model.discharge = pyo.Var(model.T, within=pyo.NonNegativeReals, bounds=(0, max_discharge_rate))
model.soc = pyo.Var(model.T, within=pyo.NonNegativeReals, bounds=(0, battery_capacity))

# Objective function: Maximize revenue
def revenue_rule(model):
    return sum(price[t] * (model.discharge[t] * discharge_efficiency - model.charge[t] / charge_efficiency) for t in model.T)
model.revenue = pyo.Objective(rule=revenue_rule, sense=pyo.maximize)

# Constraints
def soc_rule(model, t):
    if t == 0:
        return model.soc[t] == initial_soc + model.charge[t] * charge_efficiency - model.discharge[t] / discharge_efficiency
    else:
        return model.soc[t] == model.soc[t-1] + model.charge[t] * charge_efficiency - model.discharge[t] / discharge_efficiency
model.soc_constraint = pyo.Constraint(model.T, rule=soc_rule)

# Final state of charge constraint
model.final_soc_constraint = pyo.Constraint(expr=model.soc[hours-1] == final_soc)

# Solve the model
solver = pyo.SolverFactory('glpk')
results = solver.solve(model, tee=True)

# Extract results
charge_schedule = [pyo.value(model.charge[t]) for t in model.T]
discharge_schedule = [pyo.value(model.discharge[t]) for t in model.T]
soc_schedule = [pyo.value(model.soc[t]) for t in model.T]

# Convert results to DataFrame
results_df = pd.DataFrame({
    'Timestamp': price_data['Timestamp'],
    'Price': price,
    'Charge': charge_schedule,
    'Discharge': discharge_schedule,
    'SOC': soc_schedule
})


GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write C:\Users\angelmah\AppData\Local\Temp\tmp6xtwp186.glpk.raw --wglp
 C:\Users\angelmah\AppData\Local\Temp\tmpzriktukf.glpk.glp --cpxlp C:\Users\angelmah\AppData\Local\Temp\tmpax9ymct7.pyomo.lp
Reading problem data from 'C:\Users\angelmah\AppData\Local\Temp\tmpax9ymct7.pyomo.lp'...
8761 rows, 26280 columns, 35040 non-zeros
105128 lines were read
Writing problem data to 'C:\Users\angelmah\AppData\Local\Temp\tmpzriktukf.glpk.glp'...
113882 lines were written
GLPK Simplex Optimizer, v4.65
8761 rows, 26280 columns, 35040 non-zeros
Preprocessing...
8760 rows, 17519 columns, 26278 non-zeros
Scaling...
 A: min|aij| =  9.000e-01  max|aij| =  1.000e+00  ratio =  1.111e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 8760
      0: obj =   1.730808000e+02 inf =   2.889e+00 (2)
      7: obj =   1.510616272e+02 inf =   0.000e+00 (0)
* 11222: obj =   3.862880562e+04 inf 

In [9]:
model.revenue() * 20

772576.1123514409

In [21]:
695/772

0.9002590673575129

In [5]:
# Create interactive plot for charging and discharging
fig_charge_discharge = go.Figure()
fig_charge_discharge.add_trace(go.Scatter(x=results_df['Timestamp'], y=results_df['Charge'], mode='lines', name='Charge'))
fig_charge_discharge.add_trace(go.Scatter(x=results_df['Timestamp'], y=results_df['Discharge'], mode='lines', name='Discharge'))
fig_charge_discharge.update_layout(title='Charging and Discharging Schedule', xaxis_title='Time', yaxis_title='Power (kW)', legend=dict(x=0, y=1))

# Create interactive plot for state of charge
fig_soc = go.Figure()
fig_soc.add_trace(go.Scatter(x=results_df['Timestamp'], y=results_df['SOC'], mode='lines', name='State of Charge', line=dict(color='green')))
fig_soc.update_layout(title='State of Charge over Time', xaxis_title='Time', yaxis_title='State of Charge (kWh)', legend=dict(x=0, y=1))

# Show the plots
fig_charge_discharge.show()
fig_soc.show()

In [20]:
# Create Dash app
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = html.Div(children=[
    html.H1(children='BESS Revenue Simulation Dashboard'),

    html.Div(children='''
        Charging and Discharging Schedules.
    '''),

    dcc.Graph(
        id='charge-discharge-graph',
        figure=fig_charge_discharge
    ),

    html.Div(children='''
        State of Charge Over Time.
    '''),

    dcc.Graph(
        id='soc-graph',
        figure=fig_soc
    )
])

# Save the app layout as HTML
def save_dashboard_as_html(app, filename):
    from dash import Dash
    from dash.development.base_component import Component
    import flask
    from jinja2 import Template

    app.index_string = '''
    <!DOCTYPE html>
    <html>
        <head>
            <meta charset="UTF-8">
            <title>BESS Revenue Simulation Dashboard</title>
            {{css}}
        </head>
        <body>
            {{app_entry}}
            <footer>
                {{config}}
                {{scripts}}
                {{renderer}}
            </footer>
        </body>
    </html>
    '''
    with open(filename, 'w') as f:
        f.write(app.index_string)
        f.write(app._generate_layout_html(app.layout, {}, {}))

if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)

    # Save the dashboard as an HTML file
    save_dashboard_as_html(app, 'bess_dashboard.html')

InvalidIndexException: Missing items {%app_entry%}, {%config%}, {%scripts%} in index string.